# Identificação de Homólogos

In [4]:
import os
from Bio import SeqIO
from Bio.Blast import NCBIWWW, NCBIXML
from scripts_python import BLASTer

a = %pwd
wd = (a.rsplit('/',2))[0]
seq_id = "NM_001114380"


In [5]:

"""record = SeqIO.read(os.path.join( wd,"data/homologue", f"{seq_id}_ORF7a.fasta"), "fasta")

result_handle = NCBIWWW.qblast("blastn", "nr", record.format("fasta"), short_query=True, expect=0.10, word_size=7, entrez_query='all[filter] NOT "Severe acute respiratory syndrome coronavirus 2"[Organism]', )

save_file = open(os.path.join( wd,"data/reference", f"{seq_id}_ORF7a_blast.xml", "w"))
save_file.write(result_handle.read())
save_file.close()
result_handle.close()"""

'record = SeqIO.read(os.path.join( wd,"data/homologue", f"{seq_id}_ORF7a.fasta"), "fasta")\n\nresult_handle = NCBIWWW.qblast("blastn", "nr", record.format("fasta"), short_query=True, expect=0.10, word_size=7, entrez_query=\'all[filter] NOT "Severe acute respiratory syndrome coronavirus 2"[Organism]\', )\n\nsave_file = open(os.path.join( wd,"data/reference", f"{seq_id}_ORF7a_blast.xml", "w"))\nsave_file.write(result_handle.read())\nsave_file.close()\nresult_handle.close()'

## Carregar ficheiro XML da pesquisa

In [6]:
"""result_handle= open(os.path.join( wd,"data/homologue/BST2", "0069KN2K01R-Alignment.xml"))
blast_record_n = NCBIXML.read(result_handle)
result_handle.close()

print('Foram carregadas 100 seq:', len(blast_record_n.alignments))"""

result_handle= open(os.path.join( wd,"data/homologue/ITGAL/00D8F7PY016-Alignment.xml"))
blast_record_p = NCBIXML.read(result_handle)
result_handle.close()

print('Foram carregadas 100 seq:', len(blast_record_p.alignments))

Foram carregadas 100 seq: 100


## Identificar Homologos

In [8]:
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO

query_len = blast_record_p.query_length

homologue = []

print("Accession", "Identity%")
for aln in blast_record_p.alignments:
    for HSP in aln.hsps:
        if HSP.score > 80 and HSP.expect < 1.0e-10 and HSP.identities/HSP.align_length > 0.80:
            print(aln.accession,HSP.identities/HSP.align_length)
            homologue.append(SeqRecord(Seq(HSP.sbjct), aln.hit_id, "", ""))


print("Total:",len(homologue))


SeqIO.write(homologue, os.path.join(wd,"data/homologue/ITGAL", f"{seq_id}_ITGAL.fasta"), "fasta")

Accession Identity%
NP_002200 1.0
BAG36913 0.9991452991452991
XP_005255370 0.9991452991452991
CAA68747 0.9982905982905983
NP_001029341 0.9923076923076923
XP_034795955 0.9923076923076923
PNI12205 0.9923076923076923
XP_034795956 0.9914529914529915
XP_024209187 0.9914529914529915
EAW52244 0.9566639411283728
AAC31672 0.955846279640229
XP_024089734 0.9777777777777777
XP_032011574 0.9709401709401709
XP_024209188 0.976068376068376
XP_030653892 0.9675213675213675
XP_024209189 0.9675213675213675
XP_023046865 0.958974358974359
XP_023046867 0.958904109589041
XP_023046866 0.9581196581196582
XP_010386254 0.9555555555555556
EHH31590 0.9538461538461539
XP_025225979 0.9555555555555556
XP_037845568 0.9573742540494459
XP_009197106 0.9564102564102565
XP_033074013 0.9512820512820512
XP_014981430 0.9529914529914529
EHH60316 0.9511986301369864
XP_033074015 0.9511986301369864
XP_033074014 0.9504273504273504
XP_017736913 0.9512820512820512
XP_028697390 0.9253112033195021
XP_011544151 1.0
XP_034795957 0.991620

105